In [ ]:
from bs4 import BeautifulSoup as bs
import numpy as np
import pandas as pd
import re
import requests
import time

pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option('display.max_columns', None)
pd.set_option("max_colwidth", None)

In [ ]:
ump_scores_df = pd.read_csv('D:\Downloads\games (1).csv')

##drop nulls, shortened season
ump_scores_df = ump_scores_df.dropna(axis='index').sort_values('Date')
ump_scores_df = ump_scores_df.set_axis(['date', 'umpire', 'home', 'away', 'home_score', 'away_score', 'called_pitches', 'called_wrong', 'accuracy', 'consistency', 'favor', 'total_run_impact'], axis=1)
covid_2020 = ump_scores_df.date.str.contains('2020')
ump_scores_df = ump_scores_df[~covid_2020].reset_index(drop=True)

display(ump_scores_df)

In [ ]:
##cleaning of weather data
seattle_weather_df = pd.read_csv('D:\Downloads\\weather_seattle_dome.csv')
seattle_weather_df = seattle_weather_df[['DATE', 'HourlyPrecipitation', 'HourlyRelativeHumidity', 'HourlyWetBulbTemperature', 'HourlyWindSpeed']]
seattle_weather_df = seattle_weather_df.dropna(axis='index').reset_index(drop=True)

seattle_weather_df.HourlyPrecipitation = seattle_weather_df.HourlyPrecipitation.replace('T', '0.00').str.replace('s', '')
seattle_weather_df

In [ ]:
##bringing in scraped data
day_night_df = pd.read_csv('Data\\day_night_df.csv').drop(columns='Unnamed: 0').set_axis(['day_night'], axis=1)
start_time_df = pd.read_csv('Data\\start_time_df.csv').drop(columns='Unnamed: 0').set_axis(['start_time'], axis=1)
duration_df = pd.read_csv('Data\\duration_df.csv').drop(columns='Unnamed: 0').set_axis(['duration'], axis=1)
attendance_df = pd.read_csv('Data\\day_night_df.csv').drop(columns='Unnamed: 0').set_axis(['attendance'], axis=1)

day_night_df.day_night = day_night_df.day_night.str.extract(r'((?<=\d\:\d{2}</div><div>).*?(?=</div><div>))')
start_time_df.start_time = start_time_df.start_time.str.extract(r'((?<=Time: ).*?(?= Local))')
duration_df.duration = duration_df.duration.str.extract(r'((?<=n</strong>: ).*?(?=</div>))')
attendance_df.attendance = attendance_df.attendance.str.extract(r'((?<=ce</strong>: ).*?(?=</div>))')

In [ ]:
ump_scores_df['day_night'] = day_night_df.day_night
ump_scores_df['start_time'] = start_time_df.start_time
ump_scores_df['duration'] = duration_df.duration
ump_scores_df['attendance'] = attendance_df.attendance
ump_scores_df

In [ ]:
scrape2_df = ump_scores_df.loc[ump_scores_df.start_time.isnull()]
scrape2_df

In [ ]:
##2nd scrape after fixing LAA 3-letter code to ANA
scrape2_df.date = scrape2_df.date.str.replace('-', '')
scrape2_df.loc[scrape2_df.home == 'LAA', 'home'] = 'ANA'
url = 'https://www.baseball-reference.com/boxes/'

scorebox = []
start_time = []
duration = []
day_night = []
for i, j in zip(scrape2_df.date, scrape2_df.home):
    response = requests.get(url + j + '/' + j + i + '0.shtml')
    soup = bs(response.text, 'html.parser')
    print(j)
    print(i)
    print(response)
    scorebox = soup.find_all('div', {'class':'scorebox_meta'})
    start_time.append(scorebox)
    duration.append(scorebox)
    day_night.append(scorebox)
    time.sleep(3)

In [ ]:
##AFTER EACH SCRAPE RUN CODE TO REVERT CHANGES BEFORE .UPDATING INTO UMP_SCORES_DF
##REMEMBER TO RUN REGEX TO GET VALUES BEFORE UPDATING IN
##AFTER EACH SCRAPE RUN CODE TO REVERT CHANGES BEFORE .UPDATING INTO UMP_SCORES_DF

In [ ]:
scrape2_df.loc[scrape2_df.home == 'ANA', 'home'] = 'LAA'

date_list = []
for i in scrape2_df.date:
    date_list.append(i[:4] + '-' + i[4:6] + '-' + i[6:])

scrape2_df.date = date_list

In [ ]:
start_time_df = pd.DataFrame(start_time)
duration_df = pd.DataFrame(duration)
day_night_df = pd.DataFrame(day_night)

n = 0
if n == 0:
    start_time_df.to_csv('start_time_df2.csv')
    duration_df.to_csv('duration_df2.csv')
    day_night_df.to_csv('day_night_df2.csv')
    n += 1

In [ ]:
day_night_df = pd.read_csv('Data\\day_night_df2.csv').drop(columns='Unnamed: 0').set_axis(['day_night'], axis=1).set_index(scrape2_df.index)
start_time_df = pd.read_csv('Data\\start_time_df2.csv').drop(columns='Unnamed: 0').set_axis(['start_time'], axis=1).set_index(scrape2_df.index)
duration_df = pd.read_csv('Data\\duration_df2.csv').drop(columns='Unnamed: 0').set_axis(['duration'], axis=1).set_index(scrape2_df.index)
attendance_df = pd.read_csv('Data\\day_night_df2.csv').drop(columns='Unnamed: 0').set_axis(['attendance'], axis=1).set_index(scrape2_df.index)

day_night_df.day_night = day_night_df.day_night.str.extract(r'((?<=\d\:\d{2}</div><div>).*?(?=</div><div>))')
start_time_df.start_time = start_time_df.start_time.str.extract(r'((?<=Time: ).*?(?= Local))')
duration_df.duration = duration_df.duration.str.extract(r'((?<=n</strong>: ).*?(?=</div>))')
attendance_df.attendance = attendance_df.attendance.str.extract(r'((?<=ce</strong>: ).*?(?=</div>))')

scrape2_df['day_night'] = day_night_df.day_night
scrape2_df['start_time'] = start_time_df.start_time
scrape2_df['duration'] = duration_df.duration
scrape2_df['attendance'] = attendance_df.attendance

ump_scores_df.update(scrape2_df)
ump_scores_df

In [ ]:
scrape3_df = ump_scores_df.loc[ump_scores_df.start_time.isnull()]
scrape3_df

In [ ]:
covid_2020 = ump_scores_df.date.str.contains('2020')
ump_scores_df = ump_scores_df[~covid_2020].reset_index(drop=True)

filter_nulls = walmart_df.shop_name.isnull()
walmart_df = walmart_df[~filter_nulls].reset_index(drop=True)

In [ ]:
ump_scores_df.to_csv('ump_scores_df.csv')